![Cartoon of telecom customers](IMG_8811.png)


The telecommunications (telecom) sector in India is rapidly changing, with more and more telecom businesses being created and many customers deciding to switch between providers. "Churn" refers to the process where customers or subscribers stop using a company's services or products. Understanding the factors that influence keeping a customer as a client in predicting churn is crucial for telecom companies to enhance their service quality and customer satisfaction. As the data scientist on this project, you aim to explore the intricate dynamics of customer behavior and demographics in the Indian telecom sector in predicting customer churn, utilizing two comprehensive datasets from four major telecom partners: Airtel, Reliance Jio, Vodafone, and BSNL:

- `telecom_demographics.csv` contains information related to Indian customer demographics:

| Variable             | Description                                      |
|----------------------|--------------------------------------------------|
| `customer_id `         | Unique identifier for each customer.             |
| `telecom_partner `     | The telecom partner associated with the customer.|
| `gender `              | The gender of the customer.                      |
| `age `                 | The age of the customer.                         |
| `state`                | The Indian state in which the customer is located.|
| `city`                 | The city in which the customer is located.       |
| `pincode`              | The pincode of the customer's location.          |
| `registration_event` | When the customer registered with the telecom partner.|
| `num_dependents`      | The number of dependents (e.g., children) the customer has.|
| `estimated_salary`     | The customer's estimated salary.                 |

- `telecom_usage` contains information about the usage patterns of Indian customers:

| Variable   | Description                                                  |
|------------|--------------------------------------------------------------|
| `customer_id` | Unique identifier for each customer.                         |
| `calls_made` | The number of calls made by the customer.                    |
| `sms_sent`   | The number of SMS messages sent by the customer.             |
| `data_used`  | The amount of data used by the customer.                     |
| `churn`    | Binary variable indicating whether the customer has churned or not (1 = churned, 0 = not churned).|


In [21]:
# Import libraries and methods/functions
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
# Start your code here!

In [22]:
demografics_df = pd.read_csv('telecom_demographics.csv')
usage_df = pd.read_csv('telecom_usage.csv')

## Analysis

In [23]:
churn_df = pd.merge(demografics_df, usage_df, on='customer_id', validate='one_to_one')
churn_df.head()

,customer_id,telecom_partner,gender,age,state,city,pincode,registration_event,num_dependents,estimated_salary,calls_made,sms_sent,data_used,churn
0,15169,Airtel,F,26,Himachal Pradesh,Delhi,667173,2020-03-16,4,85979,75,21,4532,1
1,149207,Airtel,F,74,Uttarakhand,Hyderabad,313997,2022-01-16,0,69445,35,38,723,1
2,148119,Airtel,F,54,Jharkhand,Chennai,549925,2022-01-11,2,75949,70,47,4688,1
3,187288,Reliance Jio,M,29,Bihar,Hyderabad,230636,2022-07-26,3,34272,95,32,10241,1
4,14016,Vodafone,M,45,Nagaland,Bangalore,188036,2020-03-11,4,34157,66,23,5246,1


In [24]:
churn_df.describe()

,customer_id,age,pincode,num_dependents,estimated_salary,calls_made,sms_sent,data_used,churn
count,6500.000000,6500.000000,6500.000000,6500.000000,6500.000000,6500.000000,6500.000000,6500.000000,6500.000000
mean,121035.576923,46.108615,548955.907077,1.982308,85529.193385,49.789538,24.257846,5000.956308,0.200462
std,70353.990092,16.443712,259874.312026,1.404659,37545.639180,29.799221,14.650736,2940.611928,0.400377
min,47.000000,18.000000,100045.000000,0.000000,20001.000000,-10.000000,-5.000000,-969.000000,0.000000
25%,60125.750000,32.000000,321550.000000,1.000000,52905.000000,25.000000,12.000000,2493.750000,0.000000
50%,120470.500000,46.000000,550163.500000,2.000000,85286.500000,50.000000,25.000000,4975.500000,0.000000
75%,181420.750000,60.000000,775155.500000,3.000000,118817.500000,75.000000,37.000000,7504.250000,0.000000
max,243505.000000,74.000000,999740.000000,4.000000,149977.000000,108.000000,53.000000,10919.000000,1.000000


In [25]:
churn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6500 entries, 0 to 6499
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         6500 non-null   int64 
 1   telecom_partner     6500 non-null   object
 2   gender              6500 non-null   object
 3   age                 6500 non-null   int64 
 4   state               6500 non-null   object
 5   city                6500 non-null   object
 6   pincode             6500 non-null   int64 
 7   registration_event  6500 non-null   object
 8   num_dependents      6500 non-null   int64 
 9   estimated_salary    6500 non-null   int64 
 10  calls_made          6500 non-null   int64 
 11  sms_sent            6500 non-null   int64 
 12  data_used           6500 non-null   int64 
 13  churn               6500 non-null   int64 
dtypes: int64(9), object(5)
memory usage: 711.1+ KB


In [26]:
churn_df.isna().sum()

customer_id           0
telecom_partner       0
gender                0
age                   0
state                 0
city                  0
pincode               0
registration_event    0
num_dependents        0
estimated_salary      0
calls_made            0
sms_sent              0
data_used             0
churn                 0
dtype: int64

In [27]:
# Convert registration_event to datetime
churn_df['registration_event'] = pd.to_datetime(churn_df['registration_event'])

# Define a reference date
reference_date = pd.Timestamp('2020-01-01')

# Calculate the number of days since the reference date
churn_df['registration_event_days'] = (churn_df['registration_event'] - churn_df['registration_event'].min()).dt.days

# Drop the original registration_event column
churn_df.drop('registration_event', axis=1, inplace=True)

churn_df['registration_event_days'].sample(10)

1550     913
4275    1048
4831      79
4099    1172
47      1206
5312     516
165      274
1659     166
1931     542
1222     165
Name: registration_event_days, dtype: int64

## Preprocessing

In [28]:
X = churn_df.drop(['churn', 'customer_id'], axis=1)
y = churn_df['churn'].values

In [29]:
X_cat = X.select_dtypes(include=['object']).columns.tolist()
X_num = X.select_dtypes(include=['int64']).columns.tolist()

In [30]:
preprocessor = ColumnTransformer(
    transformers = [
        ('scaler', StandardScaler(), X_num),
        ('encoder', OneHotEncoder(), X_cat),
    ]
)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    stratify=y, 
    shuffle=True, 
    random_state=42,
    test_size=.2
)

## Define Models

In [32]:
logreg = Pipeline([
        ('preprocessor', preprocessor),
        ('logreg', LogisticRegression(random_state=42))
])
rf = Pipeline([
        ('preprocessor', preprocessor),
        ('rf', RandomForestClassifier(n_jobs=-1, random_state=42, max_depth=10, n_estimators=500))
])

# Model fitting

In [33]:
rf.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('scaler', StandardScaler(),
                                                  ['age', 'pincode',
                                                   'num_dependents',
                                                   'estimated_salary',
                                                   'calls_made', 'sms_sent',
                                                   'data_used',
                                                   'registration_event_days']),
                                                 ('encoder', OneHotEncoder(),
                                                  ['telecom_partner', 'gender',
                                                   'state', 'city'])])),
                ('rf',
                 RandomForestClassifier(max_depth=10, n_estimators=500,
                                        n_jobs=-1, random_state=42))])

In [34]:
logreg.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('scaler', StandardScaler(),
                                                  ['age', 'pincode',
                                                   'num_dependents',
                                                   'estimated_salary',
                                                   'calls_made', 'sms_sent',
                                                   'data_used',
                                                   'registration_event_days']),
                                                 ('encoder', OneHotEncoder(),
                                                  ['telecom_partner', 'gender',
                                                   'state', 'city'])])),
                ('logreg', LogisticRegression(random_state=42))])

In [35]:
logreg_pred = logreg.predict(X_test)
rf_pred = rf.predict(X_test)

## Model Assessment

### Logistic Regression

In [36]:
print(classification_report(y_test, logreg_pred))

              precision    recall  f1-score   support

           0       0.80      1.00      0.89      1039
           1       0.00      0.00      0.00       261

    accuracy                           0.80      1300
   macro avg       0.40      0.50      0.44      1300
weighted avg       0.64      0.80      0.71      1300



/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [37]:
print(confusion_matrix(y_test, logreg_pred))

[[1039    0]
 [ 261    0]]


### RandomForestClassifier

In [38]:
print(classification_report(y_test, rf_pred))

              precision    recall  f1-score   support

           0       0.80      1.00      0.89      1039
           1       0.00      0.00      0.00       261

    accuracy                           0.80      1300
   macro avg       0.40      0.50      0.44      1300
weighted avg       0.64      0.80      0.71      1300



/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [39]:
print(confusion_matrix(y_test, rf_pred))

[[1039    0]
 [ 261    0]]


## Decision

In [40]:
# if logreg.score